In [142]:
import copy
from linearmodels.iv import IV2SLS
from statsmodels.stats.outliers_influence import variance_inflation_factor
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import scipy as sc
import scipy.signal
import scipy.stats
#import sysidentpy as sipy
from scipy.fft import fft, ifft, fftshift
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from sklearn.metrics import mutual_info_score

from sysidentpy.metrics import mean_squared_error
from sysidentpy.model_structure_selection import FROLS
from sysidentpy.basis_function._basis_function import Polynomial
from sysidentpy.utils.display_results import results
from sysidentpy.utils.plotting import plot_residues_correlation, plot_results
from sysidentpy.residues.residues_correlation import compute_residues_autocorrelation, compute_cross_correlation
from IPython.display import display, HTML

plt.rc('font', size=30)          # controls default text sizes
plt.rc('axes', titlesize=25)     # fontsize of the axes title
plt.rc('axes', labelsize=25)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=25)    # fontsize of the tick labels
plt.rc('ytick', labelsize=25)    # fontsize of the tick labels
plt.rc('legend', fontsize=25)    # legend fontsize

df1 = pd.read_csv('data/U3.ASC', sep="\s+", names=['time', 'y', 'u'])
df2 = pd.read_csv('data/U1.ASC', sep="\s+", names=['time', 'y', 'u'])

# cut
df1 = df1.iloc[301:1500]
df2 = df2.iloc[301:930]

# z-normalise
joint_avg_u = np.average(df1['u'].append(df2['u']))
joint_avg_y = np.average(df1['y'].append(df2['y']))
joint_std_u = np.std(df1['u'].append(df2['u']))
joint_std_y = np.std(df1['y'].append(df2['y']))
df1['u'] = (df1['u']-joint_avg_u)/joint_std_u
df1['y'] = (df1['y']-joint_avg_y)/joint_std_y
df2['u'] = (df2['u']-joint_avg_u)/joint_std_u
df2['y'] = (df2['y']-joint_avg_y)/joint_std_y

u_train = np.asarray(df2['u']).reshape(-1, 1)
u_valid = np.asarray(df1['u']).reshape(-1, 1)
y_train = np.asarray(df2['y']).reshape(-1,1)
y_valid = np.asarray(df1['y']).reshape(-1,1)

/tmp/ipykernel_336/574422590.py:39: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joint_avg_u = np.average(df1['u'].append(df2['u']))
/tmp/ipykernel_336/574422590.py:40: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joint_avg_y = np.average(df1['y'].append(df2['y']))
/tmp/ipykernel_336/574422590.py:41: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joint_std_u = np.std(df1['u'].append(df2['u']))
/tmp/ipykernel_336/574422590.py:42: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joint_std_y = np.std(df1['y'].append(df2['y']))


# The four-step IV

# Step 1: estimate the regressors via LS

In [143]:
ylag = 20
ulag = 20
lag = max(ulag,ylag)

model_1 = FROLS(
    ylag=ylag,
    xlag=ulag,
    n_terms=ylag+ulag,
    estimator='least_squares',
    extended_least_squares=False,
    basis_function=Polynomial(degree=1) # linear
)
model_1.fit(X=u_train, y=y_train)

/home/kreljo/anaconda3/lib/python3.9/site-packages/sysidentpy/utils/deprecation.py:37: FutureWarning: Passing a string to define the estimator will rise an error in v0.4.0. 
 You'll have to use FROLS(estimator=LeastSquares()) instead. 
 The only change is that you'll have to define the estimator first instead of passing a string like 'least_squares'. 
 This change will make easier to implement new estimators and it'll improve code readability.
  warnings.warn(message, FutureWarning)


## Step 2: generate instruments with the ARX model, IV estimate coefficients theta

### -- Is it a problem that I shorten my PRBS train data for further calcualtions? --

In [144]:
df3 = pd.DataFrame({
    'u' : u_train[ulag:,0],
    'y' : y_train[ylag:,0],
    'x' : model_1._n_step_ahead_prediction(X=u_train, y=y_train[:lag], steps_ahead=len(y_train) - lag)[:, 0]
})

# chose the xlag equal to ylag, so that the zeta * fi.T is a square matrix
xlag = ylag

# adding lags to the dataset
for i in range(1, ylag+1):
    df3['y_lag'+str(i)] = df3['y'].shift(i)
for i in range(1, ulag+1):
    df3['u_lag'+str(i)] = df3['u'].shift(i)
for i in range(1, xlag+1):
    df3['x_lag'+str(i)] = df3['x'].shift(i)

# create a list of labels
lagged_u = ['u_lag'+str(i) for i in range(1, ulag+1)]
lagged_y = ['y_lag'+str(i) for i in range(1, ylag+1)]
lagged_x = ['x_lag'+str(i) for i in range(1, xlag+1)]

# dropping NA values (created due to shifting)
df3 = df3.dropna()

display(df3)

# --- calculate theta by hand ----

# phi_df = df3[lagged_y+lagged_u]
# phi = np.asarray(phi_df).reshape(-1,ylag+ulag)
#
# zeta_df = df3[lagged_x+lagged_u]
# zeta = np.asarray(zeta_df).reshape(-1,xlag+ulag)
#
# y3_df = df3[lagged_y]
# y3 = np.asarray(df3['y']).reshape(-1)
#
# theta3 = np.linalg.inv(zeta.T @ phi) @ phi.T @ y3
#
# print(pd.DataFrame(theta3))

# --------------------------------

model_iv_2 = IV2SLS(dependent=df3['y'],
                    exog=df3[lagged_u],
                    endog=df3[lagged_y],
                    instruments=df3[lagged_x]
                    )
results = model_iv_2.fit()

print(results.params)

,u,y,x,y_lag1,y_lag2,y_lag3,y_lag4,y_lag5,y_lag6,y_lag7,...,x_lag11,x_lag12,x_lag13,x_lag14,x_lag15,x_lag16,x_lag17,x_lag18,x_lag19,x_lag20
20,-1.006586,0.612893,0.612072,0.803179,0.949063,1.044206,1.063234,1.069577,1.075920,1.075920,...,1.186275,1.189650,1.188969,1.185868,1.188088,1.181587,1.182688,1.174243,1.171227,1.169091
21,-1.006586,0.384552,0.343947,0.612893,0.803179,0.949063,1.044206,1.063234,1.069577,1.075920,...,1.185783,1.186275,1.189650,1.188969,1.185868,1.188088,1.181587,1.182688,1.174243,1.171227
22,-1.006586,0.118154,0.079395,0.384552,0.612893,0.803179,0.949063,1.044206,1.063234,1.069577,...,1.179467,1.185783,1.186275,1.189650,1.188969,1.185868,1.188088,1.181587,1.182688,1.174243
23,-1.006586,-0.122873,-0.181138,0.118154,0.384552,0.612893,0.803179,0.949063,1.044206,1.063234,...,1.179852,1.179467,1.185783,1.186275,1.189650,1.188969,1.185868,1.188088,1.181587,1.182688
24,-1.006586,-0.351214,-0.418214,-0.122873,0.118154,0.384552,0.612893,0.803179,0.949063,1.044206,...,1.172074,1.179852,1.179467,1.185783,1.186275,1.189650,1.188969,1.185868,1.188088,1.181587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,-1.006586,0.828550,0.912359,0.974435,1.037863,1.056891,1.037863,0.993463,0.936378,0.872950,...,0.449426,0.324012,0.217496,0.129466,0.106042,0.154239,0.278316,0.468102,0.663840,0.801576
605,-1.006586,0.638266,0.699958,0.828550,0.974435,1.037863,1.056891,1.037863,0.993463,0.936378,...,0.581131,0.449426,0.324012,0.217496,0.129466,0.106042,0.154239,0.278316,0.468102,0.663840
606,-1.006586,0.397238,0.436752,0.638266,0.828550,0.974435,1.037863,1.056891,1.037863,0.993463,...,0.702330,0.581131,0.449426,0.324012,0.217496,0.129466,0.106042,0.154239,0.278316,0.468102
607,-1.006586,0.143525,0.173323,0.397238,0.638266,0.828550,0.974435,1.037863,1.056891,1.037863,...,0.811428,0.702330,0.581131,0.449426,0.324012,0.217496,0.129466,0.106042,0.154239,0.278316


u_lag1     0.007251
u_lag2     0.005979
u_lag3     0.022949
u_lag4     0.035372
u_lag5     0.022156
u_lag6     0.013260
u_lag7    -0.008583
u_lag8    -0.016962
u_lag9    -0.047321
u_lag10   -0.052832
u_lag11   -0.039152
u_lag12   -0.012220
u_lag13   -0.015567
u_lag14    0.037963
u_lag15    0.065006
u_lag16    0.016954
u_lag17   -0.009452
u_lag18   -0.024832
u_lag19   -0.011484
u_lag20    0.013093
y_lag1     1.474300
y_lag2    -0.768729
y_lag3     0.151201
y_lag4     0.137381
y_lag5     0.614032
y_lag6     0.016489
y_lag7    -0.387649
y_lag8    -0.803696
y_lag9     0.687497
y_lag10    0.740899
y_lag11   -2.514160
y_lag12    0.521143
y_lag13    2.229915
y_lag14    0.197833
y_lag15   -1.451819
y_lag16   -0.765871
y_lag17    0.813521
y_lag18    0.656829
y_lag19   -0.742640
y_lag20    0.192090
Name: parameter, dtype: float64


# Step 3: postulate AR model to predict intermediate vector $omega$